In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import re
from numpy import random
import pprint
import queue as que
from queue import LifoQueue
import operator
import collections

#reference to github: https://github.com/HolyChen/KdTree/blob/master/kdtree.py

#Prepocess Data

* Mounted Google Drive
* Create path to folder
* Load data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = "/content/gdrive/MyDrive/AI-Future/LocalSensitiveHashing/"

#definition of class POINT
* Definition
* Load data

In [45]:
#gen 3-d data
n = int(input())
f = open('a.txt',"w")
f.write("{} \n".format(n))

for i in range(0,n):
    l_gen = []
    for j in range(0,3):
        g = round(random.uniform(0,100),3)
        l_gen.append(g)
    if i != n - 1:
        f.write("{} {} {}".format(l_gen[0],l_gen[1],l_gen[2]))
        f.write('\n')
    else:
        f.write("{} {} {}".format(l_gen[0],l_gen[1],l_gen[2]))

f.close()

1000


In [46]:
#preprocess-data => convert to 3d vector for local hashing
# path_data = path + 'a.txt'
points = []
with open('a.txt','r',encoding='UTF-8') as f:
     n = int(f.readline())
     for i in range(n):
         line = f.readline()
         lst = re.split("\s+",line)
         if len(lst) >= 4:
            lst = lst[:-1]
         point = [round(float(x),3) for x in lst]

         points.append(point)      

#Definition of class Node

In [47]:
class Node:
      '''
      Fields:
        data: leaf_node <=> list else None
        median: median of the data in the node
        split: choose dimension to split
        left: left_child_node
        right: right_child_node
        parent: parent_node  
      '''

      data = None
      median = None
      left = None
      right = None
      parent = None
      split = -1


      def __init__(self, data = None, split = 0, median = None, left = None, right = None, parent = None):
          self.data = data
          self.split = split
          self.left = left
          self.right = right
          self.parrent = parent
          self.median = median

#Helper Function

In [48]:
def euclide_distance_split(a,b):

    dist = 0.0
    a = np.squeeze(a)
    b = np.squeeze(b)
    for i in range(len(a)):
        dist += pow(a[i] - b[i],2)

    dist = np.sqrt(dist)
    
    return dist

#KDtree definition

In [53]:
class KdTree_split:
      min_split = 1 #min_split: a leaf node contains no more than `min_split` data
      def __init__(self, all_data, dimension, min_split = 1):
          self.min_split = min_split
          self.dimension = dimension
          self.size = len(all_data)
          self.root = self.buildKdTree(all_data)

      def buildKdTree(self,all_data):
          if len(all_data) == 0:
             return None
          
          if len(all_data) <= self.min_split:
             return Node(all_data,0,all_data[0],None,None)
          
          #dimension of variable
          split_value = self.get_variance_dimension(all_data)
          left_data,median,right_data = self.split(all_data,split_value)

          node = Node([],split_value,median,self.buildKdTree(left_data),self.buildKdTree(right_data))
          if node.left is None:
             node.left.parent = node
          if node.right is None:
             node.right.parent = node
          
          return node  

      def get_variance_dimension(self,all_data):
          means = [0.0] * self.dimension
          vars = [0.0] * self.dimension

          for data in all_data:
              for i in range(self.dimension):
                  means[i] += data[i]
          
          means = [mean / len(all_data) for mean in means]
          
          for data in all_data:
              for i in range(self.dimension):
                  vars[i] += (data[i] - means[i])**2
          
          max_var = -1
          value = -1
          for i,var in enumerate(vars):
              if var > max_var:
                 max_var = var
                 value = i

          return value

      def split(self,all_data,dim):
          length = len(all_data)
          
          #k is number of element in partial data
          def min_k(data,begin,end,k):
              #partition in quicksort
              pivot = data[begin]
              left = begin
              right = end
              while left < right:
                    while left < right and data[right][dim] > pivot[dim]:
                          right -= 1
                    data[left] = data[right]
                    while left < right and data[left][dim] < pivot[dim]:
                          left += 1
                    data[right] = data[left]
              data[left] = pivot

              if left - begin + 1 == k:
                 return 
              elif left - begin + 1 < k:
                 #choose last k - (left - begin + 1) element from partial data: left + 1 -> end
                 return min_k(data,left + 1,end,k - (left - begin + 1)) 
              else:
                 #choose k element from data with slice index form begin -> left -1
                 return min_k(data,begin,left - 1,k)
          mid = int(length/2)
          min_k(all_data,0,length - 1,mid)
          return all_data[0 : mid],all_data[mid - 1],all_data[mid:]

      def find_kNN(self,data,k = 1):
          best_distance = np.inf
          dq = collections.deque()
          def find(root):
              nonlocal best_distance
              nonlocal dq
              if self.size == 0:
                 return None
              if root.left is None and root.right is None:  #leaf node
                 dist = euclide_distance_split(root.data,data)
                 if dist <= best_distance:
                    dq.append(root)
                    best_distance = dist
                    if len(dq) == k:
                       dq.popleft()
              else:
                 if data[root.split] <= root.median[root.split]:
                    good_side = root.left
                    bad_side = root.right
                 else:
                    good_side = root.right
                    bad_side = root.left
                 find(good_side)
                 if abs(data[root.split] - root.median[root.split]) <= best_distance:
                    find(bad_side)
          find(self.root)
          res = []
          for i in range(len(dq)):
              node_temp = dq.pop()
              res.append(node_temp)
              print("point{}: {} with euclide distance: {}".format(i + 1,np.squeeze(node_temp.data),euclide_distance_split(node_temp.data,data)))
              



#Main

In [55]:
model = KdTree(points,3)

In [57]:
model.find_kNN([1.2,2.3,3.4],3)

point1: [1.417 7.919 1.022] with euclide distance: 6.105336518161796
point2: [ 6.324  5.079 13.288] with euclide distance: 11.478273432881794
